In [1]:
import sys
sys.path.insert(0, '../backend')

from app.agents.state import GraphState, create_initial_state
from app.agents.coordinator import coordinator_node
from app.agents.search_node import search_node
from app.agents.grader import grader_node
from app.agents.rewriter import rewriter_node
from app.agents.answer_node import answer_node
from app.agents.graph import run_graph
from app.agents.prompts import format_coordinator_prompt

import json
from pathlib import Path

print("✅ Setup completado")
print("📦 Módulos cargados:")
print("   - GraphState, create_initial_state")
print("   - CoordinatorAgent, SearchNode, GraderAgent, RewriterAgent, AnswerNode")
print("   - run_graph (grafo completo)")

c:\Users\elsac\Documents\react-rag-multiagent-ucla\backend\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 432.85it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 539.38it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
----------

✅ Setup completado
📦 Módulos cargados:
   - GraphState, create_initial_state
   - CoordinatorAgent, SearchNode, GraderAgent, RewriterAgent, AnswerNode
   - run_graph (grafo completo)


In [2]:
from app.services.vector_store import VectorStoreService

vs = VectorStoreService()
stats = vs.get_stats()

print("📊 ESTADO DEL VECTOR STORE:")
print(f"   Total documentos: {stats['total_documents']}")
print(f"   Tipos de archivo: {stats.get('file_types', {})}")
print(f"   Fuentes únicas: {stats.get('unique_sources', 0)}")

if stats['total_documents'] > 0:
    print("\n✅ Vector store listo para usar")
else:
    print("\n❌ ERROR: Vector store vacío. Ejecutar notebook 02_embeddings.ipynb primero")

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 419.05it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


📊 ESTADO DEL VECTOR STORE:
   Total documentos: 392
   Tipos de archivo: {'pdf': 100}
   Fuentes únicas: 2

✅ Vector store listo para usar


In [3]:
print("🤖 TEST 1: COORDINATOR AGENT\n")
print("="*70)

# Crear estado inicial
test_query = "¿Qué es machine learning?"
state = create_initial_state(test_query)

print(f"Query: {test_query}")
print(f"Estado inicial: iteración {state['iteration']}, next_step: {state['next_step']}\n")

# Ejecutar coordinador
result = coordinator_node(state)

print("✅ Coordinador ejecutado")
print(f"   💭 Thought: {result.get('thought', 'N/A')[:80]}...")
print(f"   🎯 Action: {result.get('action', 'N/A')}")
print(f"   📝 Input: {result.get('action_input', 'N/A')[:80]}...")
print(f"   ➡️ Next: {result.get('next_step', 'N/A')}")

🤖 TEST 1: COORDINATOR AGENT

Query: ¿Qué es machine learning?
Estado inicial: iteración 1, next_step: coordinator

⚠️ Acción inválida detectada: . Usando 'search' por defecto.
✅ Coordinador ejecutado
   💭 Thought: ...
   🎯 Action: search
   📝 Input: ...
   ➡️ Next: search


In [4]:
print("🧪 TEST 2: MÚLTIPLES TIPOS DE CONSULTAS\n")

test_queries = [
    "¿Qué es inteligencia artificial?",
    "Hola, ¿cómo estás?",
    "Explícame eso",
    "¿Tienes información sobre neural networks?",
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*70}")
    print(f"TEST {i}: {query}")
    print('='*70)
    
    # Crear nuevo estado
    test_state = create_initial_state(query)
    
    # Ejecutar coordinador
    result = coordinator_node(test_state)
    
    # Mostrar resultado
    print(f"   💭 Thought: {result.get('thought', 'N/A')[:80]}...")
    print(f"   🎯 Action: {result.get('action', 'N/A')}")
    print(f"   📝 Input: {result.get('action_input', 'N/A')[:80]}...")
    print(f"   ➡️ Next: {result.get('next_step', 'N/A')}")

🧪 TEST 2: MÚLTIPLES TIPOS DE CONSULTAS


TEST 1: ¿Qué es inteligencia artificial?
⚠️ Acción inválida detectada: . Usando 'search' por defecto.
   💭 Thought: ...
   🎯 Action: search
   📝 Input: ...
   ➡️ Next: search

TEST 2: Hola, ¿cómo estás?
⚠️ Acción inválida detectada: . Usando 'search' por defecto.
   💭 Thought: ...
   🎯 Action: search
   📝 Input: ...
   ➡️ Next: search

TEST 3: Explícame eso
⚠️ Acción inválida detectada: . Usando 'search' por defecto.
   💭 Thought: ...
   🎯 Action: search
   📝 Input: ...
   ➡️ Next: search

TEST 4: ¿Tienes información sobre neural networks?
⚠️ Acción inválida detectada: . Usando 'search' por defecto.
   💭 Thought: ...
   🎯 Action: search
   📝 Input: ...
   ➡️ Next: search


In [5]:
from app.services.retriever import RetrieverService

print("🔍 TEST 3: SEARCH NODE\n")
print("="*70)

# Crear estado con query de búsqueda
search_state = create_initial_state("¿Qué es inteligencia artificial?")
search_state['action'] = 'search'
search_state['action_input'] = 'inteligencia artificial definición conceptos fundamentales'

# Ejecutar búsqueda
result = search_node(search_state)

print(f"✅ Búsqueda completada")
print(f"   Documentos encontrados: {len(result.get('retrieved_documents', []))}")
print(f"   Next step: {result.get('next_step')}")

# Mostrar documentos
for i, doc in enumerate(result.get('retrieved_documents', [])[:3], 1):
    print(f"\n📄 Doc {i}:")
    print(f"   Similitud: {doc.get('similarity', 0):.4f}")
    print(f"   Fuente: {doc.get('metadata', {}).get('source', 'N/A').split('\\')[-1]}")
    print(f"   Preview: {doc.get('document', '')[:150]}...")

🔍 TEST 3: SEARCH NODE

🔍 Search query: 'inteligencia artificial definición conceptos fundamentales'
📊 Resultados encontrados: 5
  ✅ Doc: sim=0.4321
  ✅ Doc: sim=0.3307
  ✅ Doc: sim=0.2917
  ✅ Doc: sim=0.2536
  ✅ Doc: sim=0.2134
✅ Búsqueda completada
   Documentos encontrados: 5
   Next step: grader

📄 Doc 1:
   Similitud: 0.4321
   Fuente: Inteligencia Artificial un enfoque moderno-Russell-Norvig.pdf
   Preview: Donde se intentará explicar por qué se considera a la inteligencia artificial un tema
digno de estudio y donde se intentará definirla con exactitud; e...

📄 Doc 2:
   Similitud: 0.3307
   Fuente: Inteligencia Artificial un enfoque moderno-Russell-Norvig.pdf
   Preview: Inteligencia Artificial
Un Enfoque Moderno
Inteligencia Artificial
Un Enfoque Moderno
Inteligencia Artificial
Un Enfoque Moderno
Russell • Norvig
Russ...

📄 Doc 3:
   Similitud: 0.2917
   Fuente: ejemplo.pdf
   Preview: --- Página 1 ---

Introducción a la Inteligencia Artificial
Maestría en Ciencias de la Computa

In [6]:
print("⚖️ TEST 4: GRADER NODE\n")
print("="*70)

# Crear estado con documentos recuperados (simulados)
grader_state = create_initial_state("¿Qué es inteligencia artificial?")
grader_state['retrieved_documents'] = [
    {
        "document": "La inteligencia artificial es el campo de estudio que busca crear sistemas capaces de realizar tareas que normalmente requieren inteligencia humana.",
        "metadata": {"source": "doc1.pdf"},
        "similarity": 0.85
    },
    {
        "document": "El clima está soleado hoy con temperaturas agradables.",
        "metadata": {"source": "doc2.pdf"},
        "similarity": 0.15
    },
    {
        "document": "Machine learning es una rama de la inteligencia artificial que permite a las computadoras aprender sin ser programadas explícitamente.",
        "metadata": {"source": "doc3.pdf"},
        "similarity": 0.75
    }
]

# Ejecutar grader
result = grader_node(grader_state)

print(f"✅ Grader completado")
print(f"   Documentos evaluados: 3")
print(f"   Documentos relevantes: {len(result.get('retrieved_documents', []))}")
print(f"   Next step: {result.get('next_step')}")
print(f"   Observation: {result.get('observation')}")

print("\n📄 Documentos que pasaron el filtro:")
for i, doc in enumerate(result.get('retrieved_documents', []), 1):
    print(f"   {i}. {doc['document'][:100]}...")

⚖️ TEST 4: GRADER NODE

✅ Grader completado
   Documentos evaluados: 3
   Documentos relevantes: 3
   Next step: answer
   Observation: Evaluados 3 documentos: 3 relevantes, 0 irrelevantes

📄 Documentos que pasaron el filtro:
   1. La inteligencia artificial es el campo de estudio que busca crear sistemas capaces de realizar tarea...
   2. El clima está soleado hoy con temperaturas agradables....
   3. Machine learning es una rama de la inteligencia artificial que permite a las computadoras aprender s...


In [7]:
print("✍️ TEST 5: REWRITER NODE\n")
print("="*70)

# Crear estado como si no hubiera docs relevantes
rewriter_state = create_initial_state("Explícame eso")
rewriter_state['action_input'] = "eso explicación"  # Query anterior que falló

# Ejecutar rewriter
result = rewriter_node(rewriter_state)

print(f"✅ Rewriter completado")
print(f"   Query original: {rewriter_state['user_query']}")
print(f"   Query anterior: {rewriter_state['action_input']}")
print(f"   Query reescrita: {result.get('action_input')}")
print(f"   Next step: {result.get('next_step')}")
print(f"   Action: {result.get('action')}")

✍️ TEST 5: REWRITER NODE

✅ Rewriter completado


KeyError: 'user_query'

In [ ]:
print("💬 TEST 6: ANSWER NODE\n")
print("="*70)

# Caso 1: Respuesta con contexto
print("--- CASO 1: Con contexto de documentos ---\n")
answer_state = create_initial_state("¿Qué es inteligencia artificial?")
answer_state['relevant_context'] = """
[Documento 1 - Russell-Norvig.pdf]
La inteligencia artificial es el campo de estudio que busca crear sistemas capaces de realizar tareas que normalmente requieren inteligencia humana, como el razonamiento, el aprendizaje y la percepción.

[Documento 2 - Introducción IA.pdf]
La IA se divide en diferentes áreas como machine learning, visión computacional, procesamiento de lenguaje natural y robótica.
"""

result1 = answer_node(answer_state)
print(f"Respuesta generada:\n{result1.get('final_answer')}\n")

# Caso 2: Respuesta directa (sin contexto)
print("\n--- CASO 2: Respuesta directa (saludo) ---\n")
answer_state2 = create_initial_state("Hola, ¿cómo estás?")
answer_state2['action'] = 'answer'
answer_state2['action_input'] = '¡Hola! Estoy aquí para ayudarte con tus preguntas sobre documentos académicos.'

result2 = answer_node(answer_state2)
print(f"Respuesta generada:\n{result2.get('final_answer')}\n")

# Caso 3: Clarificación
print("\n--- CASO 3: Solicitar aclaración ---\n")
answer_state3 = create_initial_state("Explícame eso")
answer_state3['action'] = 'clarify'
answer_state3['action_input'] = '¿A qué tema específico te refieres? ¿Podrías darme más detalles?'

result3 = answer_node(answer_state3)
print(f"Respuesta generada:\n{result3.get('final_answer')}")

💬 TEST 6: ANSWER NODE

--- CASO 1: Con contexto de documentos ---

Respuesta generada:
Según el Documento 1 - Russell-Norvig.pdf, la inteligencia artificial se define como "el campo de estudio que busca crear sistemas capaces de realizar tareas que normalmente requieren inteligencia humana, como el razonamiento, el aprendizaje y la percepción".


--- CASO 2: Respuesta directa (saludo) ---

Respuesta generada:
¡Hola! Estoy aquí para ayudarte con tus preguntas sobre documentos académicos.


--- CASO 3: Solicitar aclaración ---

Respuesta generada:
¿A qué tema específico te refieres? ¿Podrías darme más detalles?


In [ ]:
print("🚀 TEST 7: GRAFO COMPLETO - QUERY SIMPLE\n")
print("="*70)

test_query = "Hola"
print(f"Query: {test_query}\n")

# Ejecutar grafo completo
final_state = run_graph(test_query)

print("\n✅ EJECUCIÓN COMPLETADA")
print(f"\n📊 Resultado:")
print(f"   Iteraciones: {final_state['iteration']}")
print(f"   Respuesta: {final_state.get('final_answer')}")

print(f"\n📜 Traza ({len(final_state['trace'])} pasos):")
for i, step in enumerate(final_state['trace'], 1):
    print(f"   {i}. {step['agent']}: {step['action']}")

🚀 TEST 7: GRAFO COMPLETO - QUERY SIMPLE

Query: Hola


✅ EJECUCIÓN COMPLETADA

📊 Resultado:
   Iteraciones: 1
   Respuesta: ¡Hola! Estoy aquí para ayudarte a encontrar información en los documentos académicos. ¿Qué te gustaría saber?

📜 Traza (2 pasos):
   1. coordinator: answer
   2. answer: answer


In [ ]:
print("🚀 TEST 8: GRAFO COMPLETO - QUERY TÉCNICA\n")
print("="*70)

test_query = "¿Qué es inteligencia artificial?"
print(f"Query: {test_query}\n")

# Ejecutar grafo completo
final_state = run_graph(test_query)

print("\n✅ EJECUCIÓN COMPLETADA")
print(f"\n📊 Resultado:")
print(f"   Iteraciones: {final_state['iteration']}")
print(f"   Documentos recuperados: {len(final_state.get('retrieved_documents', []))}")

print(f"\n💬 Respuesta final:")
print(final_state.get('final_answer'))

print(f"\n📜 Traza completa ({len(final_state['trace'])} pasos):")
for i, step in enumerate(final_state['trace'], 1):
    print(f"\n[Paso {i}] Agent: {step['agent']}")
    print(f"  💭 Thought: {step['thought'][:80]}...")
    print(f"  🎯 Action: {step['action']}")
    print(f"  👁️ Observation: {step['observation'][:80]}...")

🚀 TEST 8: GRAFO COMPLETO - QUERY TÉCNICA

Query: ¿Qué es inteligencia artificial?

🔍 Search query: 'inteligencia artificial definición conceptos fundamentales'
📊 Resultados encontrados: 5
  ✅ Doc: sim=0.4321
  ✅ Doc: sim=0.3307
  ✅ Doc: sim=0.2917
  ✅ Doc: sim=0.2536
  ✅ Doc: sim=0.2134

✅ EJECUCIÓN COMPLETADA

📊 Resultado:
   Iteraciones: 1
   Documentos recuperados: 10

💬 Respuesta final:
Según el documento "Inteligencia Artificial un enfoque moderno-Russell-Norvig.pdf" de Stuart Russell y Peter Norvig, la inteligencia artificial se define como:

"...la capacidad de una máquina para realizar tareas que normalmente requieren la inteligencia humana, como el aprendizaje, la resolución de problemas, la toma de decisiones y la comprensión del lenguaje." (página 1)

Además, según el mismo documento, se considera a la inteligencia artificial un tema digno de estudio debido a su importancia en la resolución de problemas complejos en la inteligencia artificial.

📜 Traza completa (4 pasos):

[

In [ ]:
print("💬 TEST 9: CONVERSACIÓN MULTI-TURN\n")
print("="*70)

# Turno 1
print("\nUsuario: ¿Qué es inteligencia artificial?")
state1 = run_graph("¿Qué es inteligencia artificial?")
print(f"Sistema: {state1.get('final_answer')[:200]}...\n")

# Turno 2 - Follow-up
print("\nUsuario: ¿Y cómo se relaciona con machine learning?")
state2 = run_graph("¿Y cómo se relaciona con machine learning?")
print(f"Sistema: {state2.get('final_answer')[:200]}...\n")

print("\n✅ Conversación multi-turn completada")
print(f"   Total iteraciones turno 1: {state1['iteration']}")
print(f"   Total iteraciones turno 2: {state2['iteration']}")

💬 TEST 9: CONVERSACIÓN MULTI-TURN


Usuario: ¿Qué es inteligencia artificial?
Sistema: ¿A qué aspecto específico de la inteligencia artificial te gustaría saber más?...


Usuario: ¿Y cómo se relaciona con machine learning?
🔍 Search query: 'machine learning relación con [tema específico]'


No hay documentos para evaluar


📊 Resultados encontrados: 0
🔍 Search query: 'relación entre máquina aprendizaje y procesamiento de lenguaje natural'
📊 Resultados encontrados: 1
  ✅ Doc: sim=0.2199
Sistema: No hay información directamente relacionada entre el documento proporcionado y la pregunta del usuario sobre machine learning. El texto describe una forma de traducir un lenguaje de programación a cód...


✅ Conversación multi-turn completada
   Total iteraciones turno 1: 1
   Total iteraciones turno 2: 1


In [ ]:
print("⚠️ TEST 10: LÍMITE DE ITERACIONES\n")
print("="*70)

# Crear estado en iteración 4
limit_state = create_initial_state("Dame más información")
limit_state['iteration'] = 4

print(f"Estado inicial: Iteración {limit_state['iteration']}/5")

result = coordinator_node(limit_state)

print(f"\n✅ Resultado:")
print(f"   Action: {result['action']}")
print(f"   Should Continue: {result.get('should_continue', 'N/A')}")
print(f"   Next Step: {result['next_step']}")

# Probar en iteración 5 (máximo)
print(f"\n{'='*70}")
print("Probando con iteración máxima (5):")
limit_state['iteration'] = 5
result_max = coordinator_node(limit_state)

print(f"\n🛑 En iteración máxima (5):")
print(f"   Action: {result_max['action']}")
print(f"   Should Continue: {result_max.get('should_continue', 'N/A')}")
print(f"   Thought: {result_max.get('thought', 'N/A')}")

⚠️ TEST 10: LÍMITE DE ITERACIONES

Estado inicial: Iteración 4/5

✅ Resultado:
   Action: clarify
   Should Continue: True
   Next Step: answer

Probando con iteración máxima (5):


Máximo de iteraciones alcanzado



🛑 En iteración máxima (5):
   Action: answer
   Should Continue: False
   Thought: Máximo de iteraciones alcanzado


In [ ]:
import json
from pathlib import Path

print("📁 EXPORTANDO TRAZA A JSON\n")
print("="*70)

# Ejecutar ejemplo
test_query = "¿Qué es deep learning?"
final_state = run_graph(test_query)

# Exportar traza
output_dir = Path("../data/traces")
output_dir.mkdir(parents=True, exist_ok=True)

trace_file = output_dir / "coordinator_trace.json"

trace_data = {
    "query": final_state['user_query'],
    "timestamp": final_state['timestamp'],
    "iterations": final_state['iteration'],
    "final_answer": final_state.get('final_answer', ''),
    "trace": final_state['trace']
}

with open(trace_file, 'w', encoding='utf-8') as f:
    json.dump(trace_data, f, indent=2, ensure_ascii=False)

print(f"✅ Traza exportada a: {trace_file}")
print(f"\n📊 Resumen:")
print(f"   Query: {trace_data['query']}")
print(f"   Iteraciones: {trace_data['iterations']}")
print(f"   Pasos registrados: {len(trace_data['trace'])}")

📁 EXPORTANDO TRAZA A JSON

🔍 Search query: 'deep learning definición conceptos fundamentales'


No hay documentos para evaluar


📊 Resultados encontrados: 0
🔍 Search query: 'aplicaciones y principios básicos del aprendizaje profundo en inteligencia artificial'
📊 Resultados encontrados: 5
  ✅ Doc: sim=0.4880
  ✅ Doc: sim=0.4033
  ✅ Doc: sim=0.3230
  ✅ Doc: sim=0.2975
  ✅ Doc: sim=0.2404
✅ Traza exportada a: ..\data\traces\coordinator_trace.json

📊 Resumen:
   Query: ¿Qué es deep learning?
   Iteraciones: 1
   Pasos registrados: 7


In [ ]:
print("📝 VISUALIZAR PROMPT DEL COORDINADOR\n")
print("="*70)

# Crear estado de ejemplo
state = create_initial_state("¿Qué es machine learning?")

# Generar prompt
prompt = format_coordinator_prompt(state)

print(prompt)
print("="*70)

📝 VISUALIZAR PROMPT DEL COORDINADOR

Eres el COORDINADOR de un sistema RAG multiagente especializado en análisis de documentos académicos.

# TU ROL
Analizar cada consulta del usuario y decidir la mejor estrategia de acción.

# ACCIONES DISPONIBLES

## 1. search
Buscar información en la base de conocimiento vectorial.
**Usar cuando:**
- Usuario pregunta sobre contenido específico de documentos
- Hay términos técnicos o conceptos que requieren fundamento
- Necesitas datos, definiciones o explicaciones detalladas

**Generas:** Query optimizada para búsqueda semántica

## 2. answer
Responder directamente sin buscar más información.
**Usar cuando:**
- Ya tienes suficiente contexto de búsquedas previas
- Pregunta general que no requiere documentos específicos
- Saludo, agradecimiento o pregunta meta sobre el sistema
- Puedes sintetizar información ya recuperada

**Generas:** Respuesta completa al usuario

## 3. clarify
Solicitar más información al usuario.
**Usar cuando:**
- Consulta ambigu

In [ ]:
print("📊 RESUMEN DEL NOTEBOOK\n")
print("="*70)

print("""
✅ TESTS COMPLETADOS:

1. ✅ Coordinator Agent - Análisis de queries y selección de acciones
2. ✅ Múltiples tipos de consultas - Técnicas, saludos, vagas
3. ✅ Search Node - Recuperación de documentos con ChromaDB
4. ✅ Grader Node - Evaluación de relevancia de documentos
5. ✅ Rewriter Node - Reescritura de queries fallidas
6. ✅ Answer Node - Generación de respuestas finales
7. ✅ Grafo completo (query simple) - Flujo end-to-end
8. ✅ Grafo completo (query técnica) - Flujo con búsqueda
9. ✅ Conversación multi-turn - Contexto persistente
10. ✅ Límite de iteraciones - Control de bucles infinitos
11. ✅ Exportar traza JSON - Persistencia de resultados

🎯 SISTEMA RAG MULTIAGENTE FUNCIONAL

Componentes integrados:
- GraphState (gestión de estado)
- CoordinatorAgent (ReAct pattern)
- SearchNode (retrieval con ChromaDB)
- GraderAgent (filtrado de relevancia)
- RewriterAgent (optimización de queries)
- AnswerNode (generación final con Ollama)
- LangGraph (orquestación completa)

Listo para:
- Integración con API FastAPI
- Frontend React
- Deploy en producción
""")

print("="*70)

📊 RESUMEN DEL NOTEBOOK


✅ TESTS COMPLETADOS:

1. ✅ Coordinator Agent - Análisis de queries y selección de acciones
2. ✅ Múltiples tipos de consultas - Técnicas, saludos, vagas
3. ✅ Search Node - Recuperación de documentos con ChromaDB
4. ✅ Grader Node - Evaluación de relevancia de documentos
5. ✅ Rewriter Node - Reescritura de queries fallidas
6. ✅ Answer Node - Generación de respuestas finales
7. ✅ Grafo completo (query simple) - Flujo end-to-end
8. ✅ Grafo completo (query técnica) - Flujo con búsqueda
9. ✅ Conversación multi-turn - Contexto persistente
10. ✅ Límite de iteraciones - Control de bucles infinitos
11. ✅ Exportar traza JSON - Persistencia de resultados

🎯 SISTEMA RAG MULTIAGENTE FUNCIONAL

Componentes integrados:
- GraphState (gestión de estado)
- CoordinatorAgent (ReAct pattern)
- SearchNode (retrieval con ChromaDB)
- GraderAgent (filtrado de relevancia)
- RewriterAgent (optimización de queries)
- AnswerNode (generación final con Ollama)
- LangGraph (orquestación complet